# Looking at the Dataset
The purpose of this notebook is to look closer at the dataset of genes, natural language descriptions, and ontology term annotations that are used in this work. As included in the preprocessing notebooks, these data are drawn from files from either publications supplements like Oellrich, Walls et al. (2015) or model species databases such as TAIR, MaizeGDB, and SGN. The datasets are already loaded and merged using classe available through the oats package.

In [1]:
import datetime
import nltk
import pandas as pd
import numpy as np
import time
import math
import sys
import gensim
import os
import warnings
from collections import defaultdict
from nltk.corpus import brown
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string, remove_stopwords
from gensim.utils import simple_preprocess

sys.path.append("../../oats")
from oats.utils.utils import save_to_pickle, load_from_pickle, merge_list_dicts, flatten, to_hms
from oats.biology.dataset import Dataset
from oats.biology.groupings import Groupings
from oats.biology.relationships import ProteinInteractions, AnyInteractions

warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [2]:
data = load_from_pickle("../data/pickles/gene_phenotype_dataset_all_text_and_annotations_unmerged.pickle")
data.to_pandas().head()
data.describe()

,species,num_genes,unique_descriptions
0,ath,628658,9110
1,gmx,156,49
2,mtr,342,155
3,osa,772,389
4,sly,786,314
5,zma,6526,998
6,total,637240,11015


In [3]:
data = load_from_pickle("../data/pickles/gene_phenotype_dataset_all_text_and_annotations.pickle")
data.filter_has_description()
data.to_pandas().head()
data.describe()

,species,num_genes,unique_descriptions
0,ath,6364,3813
1,gmx,30,24
2,mtr,37,36
3,osa,92,85
4,sly,70,70
5,zma,1406,811
6,total,7999,4839


### What's there for each species?
The previously loaded dataset contains all of the genes that across six plant species that have natural language description data for phenotype(s) related to that gene. Each gene can have multiple descriptions annotated to it, which were combined or concatenated when the datasets from multiple sources were merged in creating the pickled datasets. Arabidopsis has the highest number of genes that satisfy this criteria, followed by maize, and then followed by the other four species which have a relatively low number of genes that satisfy this criteria, atleast given the sources used for this work. Note that the number of unique descriptions is lower than the number of genes in call cases, because multiple genes can have the same phenotype description associated with them.

In [4]:
token_lists = {}
stems_lists = {}

# For each individual species.
for species in data.get_species():
    df = data.to_pandas()
    subset = df[df["species"]==species]
    descriptions_not_stemmed = [simple_preprocess(d) for d in subset["description"].values]
    descriptions_stemmed = [preprocess_string(d) for d in subset["description"].values]
    token_lists[species] = flatten(descriptions_not_stemmed)
    stems_lists[species] = flatten(descriptions_stemmed)    
    
    po = len([t for t in subset["term_ids"].values if "PO" in t])
    go = len([t for t in subset["term_ids"].values if "GO" in t])
    
# For the entire dataset including all of the species.
df = data.to_pandas()
subset = df
descriptions_not_stemmed = [simple_preprocess(d) for d in subset["description"].values]
descriptions_stemmed = [preprocess_string(d) for d in subset["description"].values]
token_lists["total"] = flatten(descriptions_not_stemmed)
stems_lists["total"] = flatten(descriptions_stemmed)    
    
df = data.describe() 
df["total_words"] = df["species"].map(lambda x: len(token_lists[x]))
df["unique_words"] = df["species"].map(lambda x: len(set(token_lists[x])))
df["unique_stems"] = df["species"].map(lambda x: len(set(stems_lists[x])))
df

,species,num_genes,unique_descriptions,total_words,unique_words,unique_stems
0,ath,6364,3813,264189,7085,5116
1,gmx,30,24,233,81,68
2,mtr,37,36,2672,718,516
3,osa,92,85,3887,826,586
4,sly,70,70,1810,577,438
5,zma,1406,811,50029,1846,1317
6,total,7999,4839,322820,8043,5802


### What about the ontology term annations for each species?

In [5]:
# How many of the genes in this dataset for each species are mapped to atleast one term from a given ontology?
num_mapped_go = {}
num_mapped_po = {}
for species in data.get_species():
    d = data.to_pandas()
    subset = d[d["species"]==species]    
    num_mapped_po[species] = len([t for t in subset["term_ids"].values if "PO" in t])
    num_mapped_go[species] = len([t for t in subset["term_ids"].values if "GO" in t])
num_mapped_go["total"] = sum(list(num_mapped_go.values()))   
num_mapped_po["total"] = sum(list(num_mapped_po.values()))
df["go"] = df["species"].map(lambda x: num_mapped_go[x])
df["po"] = df["species"].map(lambda x: num_mapped_po[x])
df

,species,num_genes,unique_descriptions,total_words,unique_words,unique_stems,go,po
0,ath,6364,3813,264189,7085,5116,5434,4420
1,gmx,30,24,233,81,68,28,27
2,mtr,37,36,2672,718,516,30,32
3,osa,92,85,3887,826,586,46,92
4,sly,70,70,1810,577,438,23,65
5,zma,1406,811,50029,1846,1317,184,111
6,total,7999,4839,322820,8043,5802,5745,4747


### What about the biologically relevant groups like biochemical pathways and phenotypes?

In [11]:
# What are the groupings that we're interested in mapping to?
kegg_pathways_filename = "../data/pickles/groupings_from_kegg_pathways.pickle" 
pmn_pathways_filename = "../data/pickles/groupings_from_pmn_pathways.pickle"                        
lloyd_subsets_filename = "../data/pickles/groupings_from_lloyd_subsets.pickle"                     
groupings_dict = {"kegg":kegg_pathways_filename,"plantcyc":pmn_pathways_filename,"lloyd":lloyd_subsets_filename}

for name,filename in groupings_dict.items():
    groups = load_from_pickle(filename)
    id_to_group_ids, group_id_to_ids = groups.get_groupings_for_dataset(data)
    group_mapped_ids = [k for (k,v) in id_to_group_ids.items() if len(v)>0]
    species_dict = data.get_species_dictionary()
    num_mapped = {}
    for species in data.get_species():
        inputt = len([x for x in data.get_ids() if species_dict[x]==species]) 
        num_mapped[species] = len([x for x in group_mapped_ids if species_dict[x]==species])
    num_mapped["total"] = sum(list(num_mapped.values()))    
    df[name] = df["species"].map(lambda x: num_mapped[x])  
df.to_csv("~/Desktop/a.csv")
df


,species,num_genes,unique_descriptions,total_words,unique_words,unique_stems,go,po,kegg,plantcyc,lloyd
0,ath,6364,3813,264189,7085,5116,5434,4420,1572,926,2868
1,gmx,30,24,233,81,68,28,27,1,0,0
2,mtr,37,36,2672,718,516,30,32,0,2,0
3,osa,92,85,3887,826,586,46,92,0,3,0
4,sly,70,70,1810,577,438,23,65,17,2,0
5,zma,1406,811,50029,1846,1317,184,111,156,133,0
6,total,7999,4839,322820,8043,5802,5745,4747,1746,1066,2868


### How do the vocabularies used for different species compare?
One of the things we are interested in is discovering or recovering phenotype similarity between different species in order to identify phenologs (phenotypes between species that share some underlying genetic cause). For this reason, we are interested in how the vocabularies used to describe phenotypes between different species vary, because this will impact how feasible it is to use a dataset like this to identify phenologs. Because the Arabidopsis and maize datasets are the largest in this case, we will compare the vocabularies used in describing the phenotypes associated with the genes from these species in this dataset.

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
vocabs = {s:set(token_list) for s,token_list in token_lists.items()}
fdist_zma = FreqDist(token_lists["zma"])
fdist_ath = FreqDist(token_lists["ath"])
union_vocab = vocabs["zma"].union(vocabs["ath"])
table = pd.DataFrame({"token":list(union_vocab)})
stops = set(stopwords.words('english'))
table = table[~table.token.isin(stops)]
table["part_of_speech"] = table["token"].map(lambda x: nltk.pos_tag([x])[0][1][:2])
table["ath_freq"] = table["token"].map(lambda x: fdist_ath[x])
table["ath_rate"] = table["ath_freq"]*100/len(token_lists["ath"])
table["zma_freq"] = table["token"].map(lambda x: fdist_zma[x])
table["zma_rate"] = table["zma_freq"]*100/len(token_lists["zma"])
table["diff"] = table["ath_rate"]-table["zma_rate"]
table.head()

,token,part_of_speech,ath_freq,ath_rate,zma_freq,zma_rate,diff
0,approach,NN,4,0.001514,0,0.000000,0.001514
1,spike,NN,0,0.000000,7,0.013992,-0.013992
2,transmit,NN,8,0.003028,0,0.000000,0.003028
3,severity,NN,34,0.012870,0,0.000000,0.012870
4,sectoring,VB,1,0.000379,0,0.000000,0.000379


In [8]:
# What are the tokens more frequently used for Arabidopsis than maize?
table.sort_values(by="diff", ascending=False, inplace=True)
table.head(30)

,token,part_of_speech,ath_freq,ath_rate,zma_freq,zma_rate,diff
2461,embryo,NN,4410,1.669260,146,0.291831,1.377429
5126,mutant,NN,3504,1.326323,43,0.085950,1.240373
3072,phenotype,NN,3316,1.255162,53,0.105939,1.149223
1774,wild,NN,2456,0.929637,7,0.013992,0.915646
770,type,NN,2490,0.942507,14,0.027984,0.914523
1672,defective,JJ,3341,1.264625,285,0.569670,0.694955
5800,reduced,VB,2860,1.082558,216,0.431750,0.650809
5265,root,NN,1862,0.704798,42,0.083951,0.620847
2268,plants,NN,2293,0.867939,146,0.291831,0.576109
3268,growth,NN,1830,0.692686,79,0.157908,0.534778


In [9]:
# What are the tokens more frequently used for maize than Arabidopsis?
table.sort_values(by="diff", ascending=True, inplace=True)
table.head(30)

,token,part_of_speech,ath_freq,ath_rate,zma_freq,zma_rate,diff
3786,endosperm,NN,124,0.046936,1078,2.154750,-2.107814
5981,seedling,VB,636,0.240737,925,1.848928,-1.608191
3402,yellow,NN,304,0.115069,775,1.549102,-1.434032
1370,kernel,NN,0,0.000000,689,1.377201,-1.377201
839,leaf,NN,1258,0.476174,922,1.842931,-1.366757
707,green,JJ,884,0.334609,779,1.557097,-1.222488
1857,white,JJ,375,0.141944,642,1.283256,-1.141312
1376,plant,NN,412,0.155949,449,0.897479,-0.741531
44,albino,NN,222,0.084031,396,0.791541,-0.707510
7263,usually,RB,46,0.017412,353,0.705591,-0.688179


In [10]:
# Is the mean absolute value of the rate differences different between the different parts of speech?
table["abs_diff"] = abs(table["diff"])
pos_table = table.groupby("part_of_speech").mean()
pos_table.sort_values(by="abs_diff", inplace=True, ascending=False)
pos_table = pos_table[["abs_diff"]]
pos_table.reset_index()

,part_of_speech,abs_diff
0,MD,0.058763
1,CD,0.027431
2,IN,0.026202
3,JJ,0.023842
4,DT,0.019077
5,RB,0.016582
6,NN,0.013349
7,VB,0.011641
8,CC,0.007617
9,WP,0.001620
